<a href="https://colab.research.google.com/github/Shreyash007/deep_learning_course/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

Import libraries

In [2]:
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import numpy as np
import pdb
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import wandb

Labelling dataset

In [4]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
labels = {0: "T-shirt", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle boot"}
lab = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
(X_train, Y_train), (x_test, y_test) = fashion_mnist.load_data()
indexes = [0,1,3,5,6,8,16,18,19,23]
images = [X_train[i] for i in indexes]
titles = ['Ankle Boot', 'T-Shirt', 'Dress', 'Pullover', 'Sneaker', 'Sandal', 'Trouser', 'Shirt', 'Coat', 'Bag']

wandb.init(project='CS-6910 A1',entity='shreyashgadgil007')
for label in range(1,10):
 wandb.log({"Fashion-MNIST images2": wandb.Image(images[label], caption=titles[label])})
 wandb.log({"Label": label})
#unable to add all dataset on wandb, check later

Label,▁▂▃▄▅▅▆▇█
Label,9


Feedforward neural network class

In [ ]:
class FeedForwdNN:
  #need to add these parameters through sweep/other methods in wandb.
  def __init__(self,i_layer_size,h_layer_size,o_layer_size,l_rate,activation_function,epochs):
    self.i_layer_size=i_layer_size
    self.h_layer_size=h_layer_size
    self.o_layer_size=o_layer_size
    self.l_rate=l_rate
    self.activation_function=activation_function
    self.epochs=epochs

